# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report

In [2]:
wine_dataset=load_wine()
wine_df=pd.DataFrame(data=np.c_[wine_dataset['data'], wine_dataset['target']],
                    columns=wine_dataset['feature_names']+['target'])
# wine_df['label']=wine_df.target.replace(dict(enumerate(wine_dataset.target_names)))
X=np.array(wine_df[['color_intensity','od280/od315_of_diluted_wines', 'proline']])
y=np.array(wine_df.iloc[:,13:14])
df=wine_df

In [3]:
for i in range(12):
    print(df.iloc[:,i:i+1].isnull().sum())
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)
df_scaled=pd.DataFrame(data_scaled, columns=df.columns)
df=df_scaled      
df

alcohol    0
dtype: int64
malic_acid    0
dtype: int64
ash    0
dtype: int64
alcalinity_of_ash    0
dtype: int64
magnesium    0
dtype: int64
total_phenols    0
dtype: int64
flavanoids    0
dtype: int64
nonflavanoid_phenols    0
dtype: int64
proanthocyanins    0
dtype: int64
color_intensity    0
dtype: int64
hue    0
dtype: int64
od280/od315_of_diluted_wines    0
dtype: int64


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0.0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0.0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0.0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0.0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.705263,0.970356,0.582888,0.510309,0.271739,0.241379,0.056962,0.735849,0.205047,0.547782,0.130081,0.172161,0.329529,1.0
174,0.623684,0.626482,0.598930,0.639175,0.347826,0.282759,0.086498,0.566038,0.315457,0.513652,0.178862,0.106227,0.336662,1.0
175,0.589474,0.699605,0.481283,0.484536,0.543478,0.210345,0.073840,0.566038,0.296530,0.761092,0.089431,0.106227,0.397290,1.0
176,0.563158,0.365613,0.540107,0.484536,0.543478,0.231034,0.071730,0.754717,0.331230,0.684300,0.097561,0.128205,0.400856,1.0


**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [4]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [5]:
skf = StratifiedKFold(n_splits=5, random_state=17, shuffle=True)
skf.get_n_splits(X, y)

5

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)

clf_lr = LogisticRegression(random_state=17, max_iter=10000, multi_class='ovr')
model_lr=clf_lr.fit(X_train, y_train)
y_predicted_lr=clf_lr.predict(X_test)
print(y_predicted_lr)
cvs_lr=cross_val_score(clf_lr, X_test, y_test, cv=3)
print(cvs_lr)
print(cvs_lr.mean())
print(model_lr.score(X_test, y_test))

[1. 1. 1. 0. 2. 1. 0. 1. 2. 0. 2. 0. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 0. 2.
 1. 1. 0. 1. 2. 1. 2. 1. 0. 0. 2. 1. 0. 1. 1. 1. 2. 1. 1. 2. 2.]
[1.         1.         0.86666667]
0.9555555555555556
0.9111111111111111


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [12]:
from sklearn.svm import SVC

In [13]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf_SVC = make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=17))
model_SVC=clf_SVC.fit(X_train, y_train)
y_predicted_SVC=clf_SVC.predict(X_test)
print(y_predicted_SVC)
cvs_SVC=cross_val_score(clf_SVC, X_test, y_test, cv=3)
print(cvs_SVC)
print(cvs_SVC.mean())
print(model_SVC.score(X_test, y_test))

[1. 1. 1. 0. 2. 2. 0. 1. 2. 0. 2. 0. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 0. 2.
 1. 1. 0. 1. 2. 1. 2. 2. 0. 0. 2. 1. 0. 1. 1. 1. 2. 1. 1. 2. 2.]
[1.         1.         0.93333333]
0.9777777777777779
0.9555555555555556


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
conf_mtrx_lr = confusion_matrix(y_test, y_predicted_lr)
print(conf_mtrx_lr)
conf_mtrx_SVC = confusion_matrix(y_test, y_predicted_SVC)
print(conf_mtrx_SVC)

[[ 9  0  0]
 [ 0 19  0]
 [ 0  4 13]]
[[ 9  0  0]
 [ 0 19  0]
 [ 0  2 15]]


### 5.
Для каждой модели выведите classification report.

In [16]:
from sklearn.metrics import classification_report
target_names = wine_dataset.target_names #['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_predicted_lr, target_names=target_names))
print(classification_report(y_test, y_predicted_SVC, target_names=target_names))

              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00         9
     class_1       0.83      1.00      0.90        19
     class_2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45

              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00         9
     class_1       0.90      1.00      0.95        19
     class_2       1.00      0.88      0.94        17

    accuracy                           0.96        45
   macro avg       0.97      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45

